In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

--- Isolation Forest ---

In [8]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score, roc_auc_score

iso_forest= IsolationForest(contamination= 0.01, random_state= 15)
iso_forest.fit(X_train[y_train== 0])
y_pred_iso_forest= iso_forest.predict(X_test)
y_pred_iso_forest= [1 if x== -1 else 0 for x in y_pred_iso_forest]
y_pred_prob_iso_forest= iso_forest.decision_function(X_test)

cm= confusion_matrix(y_test, y_pred_iso_forest)
print(classification_report(y_test, y_pred_iso_forest, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_iso_forest, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_iso_forest))
print('AUC:', roc_auc_score(y_test, y_pred_prob_iso_forest))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.39      0.06      0.10       912

    accuracy                           0.90      9429
   macro avg       0.65      0.53      0.53      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.39285714285714285
Accuracy: 0.9000954502068088
AUC: 0.27241460062331474


--- Combining One-Class Isolation Forest with Traditional Classifiers ---

In [9]:
iso_forest= IsolationForest(contamination= 0.01, random_state= 15)
iso_forest.fit(X_train[y_train== 0])

X_train_iso_forest_scores= iso_forest.decision_function(X_train).reshape(-1, 1)
X_test_iso_forest_scores= iso_forest.decision_function(X_test).reshape(-1, 1)
X_train_combined= np.hstack((X_train, X_train_iso_forest_scores))
X_test_combined= np.hstack((X_test, X_test_iso_forest_scores))

--- LogisticRegression ---

In [10]:
from sklearn.linear_model import LogisticRegression
lr_combined= LogisticRegression(max_iter= 1000, random_state= 15)
lr_combined.fit(X_train_combined, y_train)
y_pred_lr_combined = lr_combined.predict(X_test_combined)
y_pred_prob_lr_combined= lr_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_combined, labels= lr_combined.classes_)
print(classification_report(y_test, y_pred_lr_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.57      0.14      0.22       912

    accuracy                           0.91      9429
   macro avg       0.74      0.56      0.58      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5688073394495413
Accuracy: 0.9064587973273942
AUC: 0.8545074453775627


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt_combined = DecisionTreeClassifier(random_state=15)
dt_combined.fit(X_train_combined, y_train)
y_pred_dt_combined = dt_combined.predict(X_test_combined)
y_pred_prob_dt_combined= dt_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_combined, labels= dt_combined.classes_)
print(classification_report(y_test, y_pred_dt_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_combined))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      8517
           1       0.22      0.24      0.23       912

    accuracy                           0.85      9429
   macro avg       0.57      0.57      0.57      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.22142121524201855
Accuracy: 0.8459009438964895
AUC: 0.5730483692058608


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_combined= GridSearchCV(estimator= dt_combined, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_dt_combined.best_params_)
print("Best Precision Score:", gs_dt_combined.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 5, 'min_samples_split': 2}
Best Precision Score: 0.4777994971861396


In [13]:
tuned_dt_combined= gs_dt_combined.best_estimator_
y_pred_tuned_dt_combined= tuned_dt_combined.predict(X_test_combined)
y_pred_prob_tuned_dt_combined= tuned_dt_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_combined, labels= tuned_dt_combined.classes_)
print(classification_report(y_test, y_pred_tuned_dt_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.40      0.08      0.13       912

    accuracy                           0.90      9429
   macro avg       0.66      0.53      0.54      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.40331491712707185
Accuracy: 0.8995651712800933
AUC: 0.7818997582749876


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn_combined= KNeighborsClassifier()
knn_combined.fit(X_train_combined, y_train)
y_pred_knn_combined= knn_combined.predict(X_test_combined)
y_pred_prob_knn_combined= knn_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_combined, labels= knn_combined.classes_)
print(classification_report(y_test, y_pred_knn_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_combined))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.37      0.11      0.17       912

    accuracy                           0.90      9429
   macro avg       0.64      0.55      0.56      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.36619718309859156
Accuracy: 0.8952168840810266
AUC: 0.7062157933874254


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_combined= GridSearchCV(estimator= knn_combined, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_combined.fit(X_train_combined, y_train)
print("Best Parameters:", gs_knn_combined.best_params_)
print("Best Precision Score:", gs_knn_combined.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 28, 'weights': 'uniform'}
Best Precision Score: 0.6761708619374523


In [16]:
tuned_knn_combined= gs_knn_combined.best_estimator_
y_pred_tuned_knn_combined= tuned_knn_combined.predict(X_test_combined)
y_pred_prob_tuned_knn_combined= tuned_knn_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_combined, labels= tuned_knn_combined.classes_)
print(classification_report(y_test, y_pred_tuned_knn_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_combined))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.52      0.01      0.03       912

    accuracy                           0.90      9429
   macro avg       0.71      0.51      0.49      9429
weighted avg       0.87      0.90      0.86      9429

Precision: 0.52
Accuracy: 0.9033831795524446
AUC: 0.808556583942538


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb_combined= GaussianNB()
nb_combined.fit(X_train_combined, y_train)
y_pred_nb_combined= nb_combined.predict(X_test_combined)
y_pred_prob_nb_combined= nb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_combined, labels= nb_combined.classes_)
print(classification_report(y_test, y_pred_nb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_combined))

              precision    recall  f1-score   support

           0       0.97      0.70      0.81      8517
           1       0.22      0.77      0.34       912

    accuracy                           0.71      9429
   macro avg       0.59      0.73      0.57      9429
weighted avg       0.89      0.71      0.77      9429

Precision: 0.21567417463745758
Accuracy: 0.7078163113797857
AUC: 0.8080232723407674


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc_combined= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_combined.fit(X_train_combined, y_train)
y_pred_svc_combined= svc_combined.predict(X_test_combined)
y_pred_prob_svc_combined= svc_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_combined, labels= svc_combined.classes_)
print(classification_report(y_test, y_pred_svc_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_combined))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.20      0.00      0.01       912

    accuracy                           0.90      9429
   macro avg       0.55      0.50      0.48      9429
weighted avg       0.84      0.90      0.86      9429

Precision: 0.2
Accuracy: 0.9020044543429844
AUC: 0.6972400014213062


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf_combined= RandomForestClassifier(random_state=15)
rf_combined.fit(X_train_combined, y_train)
y_pred_rf_combined= rf_combined.predict(X_test_combined)
y_pred_prob_rf_combined= rf_combined.predict_proba(X_test_combined)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf_combined, labels= rf_combined.classes_)
print(classification_report(y_test, y_pred_rf_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.39      0.06      0.11       912

    accuracy                           0.90      9429
   macro avg       0.65      0.53      0.53      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.38926174496644295
Accuracy: 0.8997772828507795
AUC: 0.8248994142777397


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_combined= GridSearchCV(estimator= rf_combined, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_combined.fit(X_train_combined, y_train)
print("Best Parameters:", gs_tuned_rf_combined.best_params_)
print("Best Precision Score:", gs_tuned_rf_combined.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Best Precision Score: 0.6931029357116314


In [22]:
tuned_rf_combined= gs_tuned_rf_combined.best_estimator_
y_pred_tuned_rf_combined= tuned_rf_combined.predict(X_test_combined)
y_pred_prob_tuned_rf_combined= tuned_rf_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_combined, labels= tuned_rf_combined.classes_)
print(classification_report(y_test, y_pred_tuned_rf_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_combined))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.66      0.04      0.07       912

    accuracy                           0.90      9429
   macro avg       0.78      0.52      0.51      9429
weighted avg       0.88      0.90      0.86      9429

Precision: 0.66
Accuracy: 0.9049740163325909
AUC: 0.8479631938393595


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada_combined= AdaBoostClassifier(random_state=15)
ada_combined.fit(X_train_combined, y_train)
y_pred_ada_combined= ada_combined.predict(X_test_combined)
y_pred_prob_ada_combined= ada_combined.predict_proba(X_test_combined)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada_combined, labels= ada_combined.classes_)
print(classification_report(y_test, y_pred_ada_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_combined))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      8517
           1       0.53      0.15      0.24       912

    accuracy                           0.90      9429
   macro avg       0.72      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5285171102661597
Accuracy: 0.9048679605472478
AUC: 0.8533943464979226


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_combined= GridSearchCV(estimator= ada_combined, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_combined.fit(X_train_combined, y_train)
print("Best Parameters:", gs_ada_combined.best_params_)
print("Best Precision Score:", gs_ada_combined.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 100}
Best Precision Score: 0.5356166327094426


In [26]:
tuned_ada_combined= gs_ada_combined.best_estimator_
y_pred_tuned_ada_combined= tuned_ada_combined.predict(X_test_combined)
y_pred_prob_tuned_ada_combined= tuned_ada_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_combined, labels= tuned_ada_combined.classes_)
print(classification_report(y_test, y_pred_tuned_ada_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_combined))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      8517
           1       0.51      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.71      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5115384615384615
Accuracy: 0.90391345847916
AUC: 0.8529823737458003


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb_combined= GradientBoostingClassifier(random_state=15)
grb_combined.fit(X_train_combined, y_train)
y_pred_grb_combined= grb_combined.predict(X_test_combined)
y_pred_prob_grb_combined= grb_combined.predict_proba(X_test_combined)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_combined, labels= grb_combined.classes_)
print(classification_report(y_test, y_pred_grb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.55      0.12      0.19       912

    accuracy                           0.91      9429
   macro avg       0.73      0.55      0.57      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5520833333333334
Accuracy: 0.9053982394739633
AUC: 0.8530007837781609


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_combined= GridSearchCV(estimator= grb_combined, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_grb_combined.best_params_)
print("Best Precision Score:", gs_grb_combined.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.5655353296936242


In [29]:
tuned_grb_combined= gs_grb_combined.best_estimator_
y_pred_tuned_grb_combined= tuned_grb_combined.predict(X_test_combined)
y_pred_prob_tuned_grb_combined= tuned_grb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_combined, labels= tuned_grb_combined.classes_)
print(classification_report(y_test, y_pred_tuned_grb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_combined))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.55      0.12      0.19       912

    accuracy                           0.91      9429
   macro avg       0.73      0.55      0.57      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5520833333333334
Accuracy: 0.9053982394739633
AUC: 0.8530007837781609


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb_combined= XGBClassifier(random_state=15)
xgb_combined.fit(X_train_combined, y_train)
y_pred_xgb_combined= xgb_combined.predict(X_test_combined)
y_pred_prob_xgb_combined= xgb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_combined, labels= xgb_combined.classes_)
print(classification_report(y_test, y_pred_xgb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_combined))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.49      0.14      0.22       912

    accuracy                           0.90      9429
   macro avg       0.70      0.56      0.58      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4883720930232558
Accuracy: 0.902640789055043
AUC: 0.8366507439197972


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_combined= GridSearchCV(estimator= xgb_combined, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_xgb_combined.best_params_)
print("Best Precision Score:", gs_xgb_combined.best_score_)

tuned_xgb_combined= gs_xgb_combined.best_estimator_
y_pred_tuned_xgb_combined= tuned_xgb_combined.predict(X_test_combined)
y_pred_prob_tuned_xgb_combined= tuned_xgb_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_combined, labels= tuned_xgb_combined.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_combined))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.5229290880590447
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.51      0.09      0.16       912

    accuracy                           0.90      9429
   macro avg       0.71      0.54      0.55      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.5120481927710844
Accuracy: 0.9037013469084738
AUC: 0.8506267264233143


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm_combined= LGBMClassifier(random_state=15)
lgm_combined.fit(X_train_combined, y_train)
y_pred_lgm_combined= lgm_combined.predict(X_test_combined)
y_pred_prob_lgm_combined= lgm_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_combined, labels= lgm_combined.classes_)
print(classification_report(y_test, y_pred_lgm_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_combined))

[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009898 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 506
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096755 -> initscore=-2.233817
[LightGBM] [Info] Start training from score -2.233817
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.54      0.09      0.16       912

    accuracy                           0.90      9429
   macro avg       0.72      0.54      0.55      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.5384615384615384
Accuracy: 0.9045497931912185
AUC: 0.8505501896104591


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_combined= GridSearchCV(estimator= lgm_combined, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_lgm_combined.best_params_)
print("Best Precision Score:", gs_lgm_combined.best_score_)

tuned_lgm_combined= gs_lgm_combined.best_estimator_
y_pred_tuned_lgm_combined= tuned_lgm_combined.predict(X_test_combined)
y_pred_prob_tuned_lgm_combined= tuned_lgm_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_combined, labels= tuned_lgm_combined.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_combined))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008225 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 506
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.096755 -> initscore=-2.233817
[LightGBM] [Info] Start training from score -2.233817
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.55728354127445

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat_combined= CatBoostClassifier(random_state=15)
cat_combined.fit(X_train_combined, y_train)
y_pred_cat_combined= cat_combined.predict(X_test_combined)
y_pred_prob_cat_combined= cat_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_combined, labels= cat_combined.classes_)
print(classification_report(y_test, y_pred_cat_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_combined))

Learning rate set to 0.04854
0:	learn: 0.6401293	total: 167ms	remaining: 2m 47s
1:	learn: 0.5952381	total: 186ms	remaining: 1m 32s
2:	learn: 0.5564288	total: 204ms	remaining: 1m 7s
3:	learn: 0.5246917	total: 223ms	remaining: 55.4s
4:	learn: 0.4934666	total: 240ms	remaining: 47.8s
5:	learn: 0.4689867	total: 259ms	remaining: 42.9s
6:	learn: 0.4423395	total: 278ms	remaining: 39.5s
7:	learn: 0.4220842	total: 296ms	remaining: 36.8s
8:	learn: 0.4013030	total: 315ms	remaining: 34.7s
9:	learn: 0.3871715	total: 334ms	remaining: 33.1s
10:	learn: 0.3725733	total: 353ms	remaining: 31.8s
11:	learn: 0.3596578	total: 372ms	remaining: 30.6s
12:	learn: 0.3480112	total: 391ms	remaining: 29.7s
13:	learn: 0.3384408	total: 409ms	remaining: 28.8s
14:	learn: 0.3296327	total: 427ms	remaining: 28s
15:	learn: 0.3219040	total: 447ms	remaining: 27.5s
16:	learn: 0.3153698	total: 467ms	remaining: 27s
17:	learn: 0.3091380	total: 484ms	remaining: 26.4s
18:	learn: 0.3036219	total: 503ms	remaining: 26s
19:	learn: 0.298

167:	learn: 0.2359874	total: 3.64s	remaining: 18s
168:	learn: 0.2359114	total: 3.66s	remaining: 18s
169:	learn: 0.2357931	total: 3.68s	remaining: 18s
170:	learn: 0.2357153	total: 3.7s	remaining: 18s
171:	learn: 0.2356642	total: 3.73s	remaining: 17.9s
172:	learn: 0.2355748	total: 3.75s	remaining: 17.9s
173:	learn: 0.2354901	total: 3.77s	remaining: 17.9s
174:	learn: 0.2354345	total: 3.79s	remaining: 17.9s
175:	learn: 0.2353540	total: 3.82s	remaining: 17.9s
176:	learn: 0.2352829	total: 3.84s	remaining: 17.8s
177:	learn: 0.2352299	total: 3.86s	remaining: 17.8s
178:	learn: 0.2351632	total: 3.88s	remaining: 17.8s
179:	learn: 0.2350979	total: 3.9s	remaining: 17.8s
180:	learn: 0.2350195	total: 3.92s	remaining: 17.7s
181:	learn: 0.2349394	total: 3.94s	remaining: 17.7s
182:	learn: 0.2348461	total: 3.97s	remaining: 17.7s
183:	learn: 0.2347673	total: 4s	remaining: 17.7s
184:	learn: 0.2346797	total: 4.02s	remaining: 17.7s
185:	learn: 0.2345843	total: 4.04s	remaining: 17.7s
186:	learn: 0.2345143	tot

328:	learn: 0.2245684	total: 7.11s	remaining: 14.5s
329:	learn: 0.2245311	total: 7.13s	remaining: 14.5s
330:	learn: 0.2244753	total: 7.16s	remaining: 14.5s
331:	learn: 0.2244237	total: 7.18s	remaining: 14.4s
332:	learn: 0.2243577	total: 7.2s	remaining: 14.4s
333:	learn: 0.2243008	total: 7.22s	remaining: 14.4s
334:	learn: 0.2242364	total: 7.24s	remaining: 14.4s
335:	learn: 0.2241738	total: 7.26s	remaining: 14.3s
336:	learn: 0.2241138	total: 7.28s	remaining: 14.3s
337:	learn: 0.2240712	total: 7.3s	remaining: 14.3s
338:	learn: 0.2240299	total: 7.32s	remaining: 14.3s
339:	learn: 0.2239530	total: 7.34s	remaining: 14.3s
340:	learn: 0.2238640	total: 7.37s	remaining: 14.2s
341:	learn: 0.2237935	total: 7.39s	remaining: 14.2s
342:	learn: 0.2237364	total: 7.41s	remaining: 14.2s
343:	learn: 0.2236907	total: 7.43s	remaining: 14.2s
344:	learn: 0.2236197	total: 7.45s	remaining: 14.1s
345:	learn: 0.2235768	total: 7.47s	remaining: 14.1s
346:	learn: 0.2235191	total: 7.5s	remaining: 14.1s
347:	learn: 0.2

489:	learn: 0.2158985	total: 10.6s	remaining: 11s
490:	learn: 0.2158575	total: 10.6s	remaining: 11s
491:	learn: 0.2157988	total: 10.7s	remaining: 11s
492:	learn: 0.2157731	total: 10.7s	remaining: 11s
493:	learn: 0.2157374	total: 10.7s	remaining: 11s
494:	learn: 0.2157369	total: 10.7s	remaining: 10.9s
495:	learn: 0.2156816	total: 10.8s	remaining: 10.9s
496:	learn: 0.2156720	total: 10.8s	remaining: 10.9s
497:	learn: 0.2156151	total: 10.8s	remaining: 10.9s
498:	learn: 0.2155808	total: 10.8s	remaining: 10.9s
499:	learn: 0.2155378	total: 10.8s	remaining: 10.8s
500:	learn: 0.2154957	total: 10.9s	remaining: 10.8s
501:	learn: 0.2154079	total: 10.9s	remaining: 10.8s
502:	learn: 0.2153615	total: 10.9s	remaining: 10.8s
503:	learn: 0.2153010	total: 10.9s	remaining: 10.8s
504:	learn: 0.2152082	total: 10.9s	remaining: 10.7s
505:	learn: 0.2151485	total: 11s	remaining: 10.7s
506:	learn: 0.2151116	total: 11s	remaining: 10.7s
507:	learn: 0.2150812	total: 11s	remaining: 10.7s
508:	learn: 0.2150144	total:

649:	learn: 0.2081082	total: 14.3s	remaining: 7.7s
650:	learn: 0.2080555	total: 14.3s	remaining: 7.68s
651:	learn: 0.2080180	total: 14.3s	remaining: 7.66s
652:	learn: 0.2079860	total: 14.4s	remaining: 7.63s
653:	learn: 0.2079462	total: 14.4s	remaining: 7.61s
654:	learn: 0.2079063	total: 14.4s	remaining: 7.59s
655:	learn: 0.2078659	total: 14.4s	remaining: 7.57s
656:	learn: 0.2078309	total: 14.5s	remaining: 7.55s
657:	learn: 0.2077890	total: 14.5s	remaining: 7.53s
658:	learn: 0.2077275	total: 14.5s	remaining: 7.51s
659:	learn: 0.2076895	total: 14.5s	remaining: 7.48s
660:	learn: 0.2076554	total: 14.5s	remaining: 7.46s
661:	learn: 0.2076353	total: 14.6s	remaining: 7.44s
662:	learn: 0.2075939	total: 14.6s	remaining: 7.42s
663:	learn: 0.2075315	total: 14.6s	remaining: 7.39s
664:	learn: 0.2074803	total: 14.6s	remaining: 7.37s
665:	learn: 0.2074373	total: 14.7s	remaining: 7.35s
666:	learn: 0.2073855	total: 14.7s	remaining: 7.33s
667:	learn: 0.2073419	total: 14.7s	remaining: 7.31s
668:	learn: 0

813:	learn: 0.2007217	total: 18s	remaining: 4.11s
814:	learn: 0.2006869	total: 18s	remaining: 4.09s
815:	learn: 0.2006449	total: 18s	remaining: 4.06s
816:	learn: 0.2005817	total: 18s	remaining: 4.04s
817:	learn: 0.2005432	total: 18.1s	remaining: 4.02s
818:	learn: 0.2005071	total: 18.1s	remaining: 4s
819:	learn: 0.2004739	total: 18.1s	remaining: 3.98s
820:	learn: 0.2004306	total: 18.1s	remaining: 3.95s
821:	learn: 0.2003737	total: 18.2s	remaining: 3.93s
822:	learn: 0.2003508	total: 18.2s	remaining: 3.91s
823:	learn: 0.2003121	total: 18.2s	remaining: 3.89s
824:	learn: 0.2002796	total: 18.2s	remaining: 3.86s
825:	learn: 0.2002308	total: 18.2s	remaining: 3.84s
826:	learn: 0.2001793	total: 18.3s	remaining: 3.82s
827:	learn: 0.2001440	total: 18.3s	remaining: 3.8s
828:	learn: 0.2001018	total: 18.3s	remaining: 3.77s
829:	learn: 0.2000687	total: 18.3s	remaining: 3.75s
830:	learn: 0.2000555	total: 18.3s	remaining: 3.73s
831:	learn: 0.2000288	total: 18.4s	remaining: 3.71s
832:	learn: 0.1999887	to

975:	learn: 0.1937966	total: 22.4s	remaining: 550ms
976:	learn: 0.1937595	total: 22.4s	remaining: 527ms
977:	learn: 0.1937236	total: 22.4s	remaining: 504ms
978:	learn: 0.1936701	total: 22.5s	remaining: 482ms
979:	learn: 0.1936196	total: 22.5s	remaining: 459ms
980:	learn: 0.1935961	total: 22.5s	remaining: 436ms
981:	learn: 0.1935690	total: 22.5s	remaining: 413ms
982:	learn: 0.1935289	total: 22.6s	remaining: 390ms
983:	learn: 0.1934697	total: 22.6s	remaining: 368ms
984:	learn: 0.1934215	total: 22.6s	remaining: 345ms
985:	learn: 0.1933846	total: 22.7s	remaining: 322ms
986:	learn: 0.1933560	total: 22.7s	remaining: 299ms
987:	learn: 0.1933153	total: 22.7s	remaining: 276ms
988:	learn: 0.1932779	total: 22.8s	remaining: 253ms
989:	learn: 0.1932299	total: 22.8s	remaining: 230ms
990:	learn: 0.1931989	total: 22.8s	remaining: 207ms
991:	learn: 0.1931507	total: 22.9s	remaining: 184ms
992:	learn: 0.1931114	total: 22.9s	remaining: 161ms
993:	learn: 0.1930820	total: 22.9s	remaining: 138ms
994:	learn: 

In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_combined= GridSearchCV(estimator= cat_combined, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_combined.fit(X_train_combined, y_train)

print("Best Parameters:", gs_cat_combined.best_params_)
print("Best Precision Score:", gs_cat_combined.best_score_)

tuned_cat_combined= gs_cat_combined.best_estimator_
y_pred_tuned_cat_combined= tuned_cat_combined.predict(X_test_combined)
y_pred_prob_tuned_cat_combined= tuned_cat_combined.predict_proba(X_test_combined)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_combined, labels= tuned_cat_combined.classes_)
print(classification_report(y_test, y_pred_tuned_cat_combined, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_combined, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_combined))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_combined))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6810789	total: 80.4ms	remaining: 723ms
1:	learn: 0.6693369	total: 164ms	remaining: 657ms
2:	learn: 0.6584132	total: 249ms	remaining: 580ms
3:	learn: 0.6472170	total: 286ms	remaining: 429ms
4:	learn: 0.6379745	total: 363ms	remaining: 363ms
5:	learn: 0.6289233	total: 443ms	remaining: 295ms
6:	learn: 0.6184043	total: 523ms	remaining: 224ms
7:	learn: 0.6086066	total: 575ms	remaining: 144ms
8:	learn: 0.5996223	total: 607ms	remaining: 67.4ms
9:	learn: 0.5911849	total: 644ms	remaining: 0us
Best Parameters: {'depth': 10, 'iterations': 10, 'learning_rate': 0.01}
Best Precision Score: 0.7761904761904762
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.75      0.01      0.02       912

    accuracy                           0.90      9429
   macro avg       0.83      0.50      0.48      9429
weighted avg       0.89      0.90      0.86      9429

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_combined, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_combined, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_combined, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_combined, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_combined, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_combined, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_combined, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_combined, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_combined, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_combined, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_combined, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_combined, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_combined, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_combined, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_combined, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_combined, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_combined, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_combined, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_combined, zero_division= 0)
                  }

combined_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
combined_precision= combined_precision.sort_values(by= 'Precision Score', ascending=False)
print(combined_precision)

                                    Model  Precision Score
18              Tuned CatBoost Precision:         0.750000
8          Tuned Random Forest Precision:         0.660000
0          Logistic Regression Precision:         0.568807
16                  Tuned LGBM Precision:         0.554140
11            GradientBoosting Precision:         0.552083
12      Tuned GradientBoosting Precision:         0.552083
17                    CatBoost Precision:         0.541667
15                        LGBM Precision:         0.538462
9                     AdaBoost Precision:         0.528517
4   Tuned KNeighborsClassifier Precision:         0.520000
14                   Tuned XGB Precision:         0.512048
10              Tuned AdaBoost Precision:         0.511538
13                         XGB Precision:         0.488372
2          Tuned Decision Tree Precision:         0.403315
7                Random Forest Precision:         0.389262
3         KNeighborsClassifier Precision:         0.3661

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_combined),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_combined),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_combined),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_combined),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_combined),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_combined),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_combined),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_combined),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_combined),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_combined),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_combined),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_combined),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_combined),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_combined),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_combined),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_combined),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_combined),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_combined),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_combined)
                  }

combined_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
combined_accuracy= combined_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(combined_accuracy)

                                   Model  Accuracy Score
0          Logistic Regression Accuracy:        0.906459
12      Tuned GradientBoosting Accuracy:        0.905398
11            GradientBoosting Accuracy:        0.905398
16                  Tuned LGBM Accuracy:        0.905080
8          Tuned Random Forest Accuracy:        0.904974
17                    CatBoost Accuracy:        0.904974
9                     AdaBoost Accuracy:        0.904868
15                        LGBM Accuracy:        0.904550
10              Tuned AdaBoost Accuracy:        0.903913
18              Tuned CatBoost Accuracy:        0.903913
14                   Tuned XGB Accuracy:        0.903701
4   Tuned KNeighborsClassifier Accuracy:        0.903383
13                         XGB Accuracy:        0.902641
6                          SVM Accuracy:        0.902004
7                Random Forest Accuracy:        0.899777
2          Tuned Decision Tree Accuracy:        0.899565
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_combined),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_combined),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_combined),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_combined),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_combined),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_combined),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_combined),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_combined),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_combined),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_combined),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_combined),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_combined),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_combined),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_combined),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_combined),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_combined),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_combined),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_combined),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_combined)
                  }

combined_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
combined_auc= combined_auc.sort_values(by= 'AUC Score', ascending=False)
print(combined_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.854507
9                     AdaBoost AUC:   0.853394
12      Tuned GradientBoosting AUC:   0.853001
11            GradientBoosting AUC:   0.853001
10              Tuned AdaBoost AUC:   0.852982
16                  Tuned LGBM AUC:   0.851162
14                   Tuned XGB AUC:   0.850627
15                        LGBM AUC:   0.850550
17                    CatBoost AUC:   0.849175
8          Tuned Random Forest AUC:   0.847963
13                         XGB AUC:   0.836651
7                Random Forest AUC:   0.824899
18              Tuned CatBoost AUC:   0.824327
4   Tuned KNeighborsClassifier AUC:   0.808557
5                   GaussianNB AUC:   0.808023
2          Tuned Decision Tree AUC:   0.781900
3         KNeighborsClassifier AUC:   0.706216
6                          SVM AUC:   0.697240
1                Decision Tree AUC:   0.573048
